### 使用ChatGLM实现多轮对话机器人

我们所看到的大语言模型，当调用它实现问答的时候，实际上是无状态的，模型本身不会记得你之前问的问题以及回答的内容。

但是，我们也有变通的方式，就是将历史的聊天内容，以上下文的方式，和问题一起发送给模型来生成答案，模型自然就知道了之前的聊天内容。为了实现这样的多轮对话，最简单的方式就是我们自己将历史聊天内容进行拼接即可。但是，LangChain 以及帮我们实现好了。

In [ ]:
%pip install langchain

In [3]:
from langchain import __version__
print(__version__)

0.0.235


首先我们使用 LangChain 框架，生成一个 *SagemakerEndpoint* 类型的 model 对象：

In [4]:
from typing import Dict
from langchain.prompts import PromptTemplate
from langchain import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
import json

import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.WARN)

class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        input = {"ask": prompt, **model_kwargs}
        logger.info("prompt: %s", prompt)
        logger.info("model_kwargs: %s", model_kwargs)
        input_str = json.dumps(input)
        return input_str.encode('utf-8')
    
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        logger.info("response_json: %s", response_json)
        return response_json["answer"]

content_handler = ContentHandler()

In [5]:
sagemaker_endpoint_name = "mt-chatglm2-6b-g4dn"

chatglm_model = SagemakerEndpoint(
    endpoint_name=sagemaker_endpoint_name, 
    region_name="us-east-1", 
    model_kwargs={"temperature": 0.1},
    content_handler=content_handler
)

在 LangChain 中，帮我们实现了各种 Memory 对象，用于对聊天历史记录进行管理。然后，在创建 Chain 的时候，只要传递这个 memory 对象，我们的问答机器人就具备了记忆能力。

In [6]:
from langchain.memory import ConversationBufferWindowMemory
from langchain import LLMChain

template = """You are a chatbot having a conversation with a human.

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], 
    template=template
)
conversation_memory = ConversationBufferWindowMemory(memory_key="chat_history", k=3)
conversation_chain = LLMChain(
    llm=chatglm_model,
    memory=conversation_memory,
    prompt=prompt
)

In [7]:
# 开始前清除memory
conversation_memory.clear()

In [8]:
conversation_chain("你会做数学题吗？")

{'human_input': '你会做数学题吗？',
 'chat_history': '',
 'text': '是的，我可以做数学题。请问您有什么数学问题需要帮助吗？'}

In [9]:
conversation_chain("3+4+5等于多少")

{'human_input': '3+4+5等于多少',
 'chat_history': 'Human: 你会做数学题吗？\nAI: 是的，我可以做数学题。请问您有什么数学问题需要帮助吗？',
 'text': '3+4+5=12'}

In [10]:
conversation_chain("再加5呢？")

{'human_input': '再加5呢？',
 'chat_history': 'Human: 你会做数学题吗？\nAI: 是的，我可以做数学题。请问您有什么数学问题需要帮助吗？\nHuman: 3+4+5等于多少\nAI: 3+4+5=12',
 'text': '再加5等于17。'}

In [11]:
conversation_chain("再乘以2呢？")

{'human_input': '再乘以2呢？',
 'chat_history': 'Human: 你会做数学题吗？\nAI: 是的，我可以做数学题。请问您有什么数学问题需要帮助吗？\nHuman: 3+4+5等于多少\nAI: 3+4+5=12\nHuman: 再加5呢？\nAI: 再加5等于17。',
 'text': '再乘以2等于34。'}

In [12]:
conversation_chain("再减2呢")

{'human_input': '再减2呢',
 'chat_history': 'Human: 3+4+5等于多少\nAI: 3+4+5=12\nHuman: 再加5呢？\nAI: 再加5等于17。\nHuman: 再乘以2呢？\nAI: 再乘以2等于34。',
 'text': '再减2等于32。'}

#### 使用 Chinese LLaMA Alpaca 测试

In [10]:
#创建 LangChain SagemakerEndpoint 的模型：
alpaca_model = SagemakerEndpoint(
    endpoint_name="chinese-alpaca-plus-7b", 
    region_name="us-east-1", 
    model_kwargs={"temperature": 0.01},
    content_handler=content_handler
)

In [16]:
from langchain.memory import ConversationBufferWindowMemory
from langchain import LLMChain

template2 = """You are a chatbot having a conversation with a human. Try to answer the question in Chinese.
{chat_history}

{human_input}

### Response:"""

prompt2 = PromptTemplate(
    input_variables=["chat_history", "human_input"], 
    template=template2
)
conversation_memory2 = ConversationBufferWindowMemory(memory_key="chat_history", k=3)
conversation_chain2 = LLMChain(
    llm=alpaca_model,
    memory=conversation_memory2,
    prompt=prompt2
)

In [30]:
conversation_memory2.clear()

In [31]:
conversation_chain2("你会做数学题吗？")

{'human_input': '你会做数学题吗？', 'chat_history': '', 'text': '是的，我会做数学题。'}

In [32]:
conversation_chain2("3+4+5等于多少")

{'human_input': '3+4+5等于多少',
 'chat_history': 'Human: 你会做数学题吗？\nAI: 是的，我会做数学题。',
 'text': '3加4加5等于12'}

In [33]:
conversation_chain2("再加5呢？")

{'human_input': '再加5呢？',
 'chat_history': 'Human: 你会做数学题吗？\nAI: 是的，我会做数学题。\nHuman: 3+4+5等于多少\nAI: 3加4加5等于12',
 'text': '12 + 5 = 17'}

In [34]:
conversation_chain2("再乘以2呢？")

{'human_input': '再乘以2呢？',
 'chat_history': 'Human: 你会做数学题吗？\nAI: 是的，我会做数学题。\nHuman: 3+4+5等于多少\nAI: 3加4加5等于12\nHuman: 再加5呢？\nAI: 12 + 5 = 17',
 'text': '2 x 17 = 34'}

In [35]:
conversation_chain2("再减2呢?")

{'human_input': '再减2呢?',
 'chat_history': 'Human: 3+4+5等于多少\nAI: 3加4加5等于12\nHuman: 再加5呢？\nAI: 12 + 5 = 17\nHuman: 再乘以2呢？\nAI: 2 x 17 = 34',
 'text': '-34-2=-36'}

### 测试结果
根据测试，Chinese LLaMA Alpaca 模型和 ChatGLM 模型都能得到比较好的效果，但是有时候会出现“计算出错”。对于这种计算问题，更好的方式是使用大语言模型生成代码，然后再通过Evaluate执行代码，返回计算的结果。但是这里只是测试多轮对话的能力和实现方式。

但是13B的模型，不知道是调优的时候使用的数据的问题，还是提示词的问题，在结果中会出现很多<p>这样的字符，而使用 Chinese LLaMA Alpaca 7B的模型，效果会比较好。但是连续多轮计算之后，总是容易出现错误。如果在提示词里，针对数学计算提供专门的提示可能会好一些。

#### 不同的memory类型

刚才使用的 Memory 类型是 ConversationBufferWindowMemory，它是基于内存的、有固定窗口大小的对象。我们创建的窗口大小为3，如果对话轮数超过3轮，就会将最老的一个对话记录清除。可以从参数的 “chat_history” 看到历史记录。

很显然，这种简单粗暴的方式有时候不适用。还好 LangChain 给我们提供了多种类型的 Memory。下面来看看几种常用的 Memory：
 * ConversationBufferMemory: 将历史记录保存在Buffer中，如果对话多轮，则有可能会超出buffer大小。
 * ConversationEntityMemory: 将对话记录中的entity实例提取出来保存起来，在后续的对话中，会根据实体以及它们之间的关系作为上下文，回答用户的问题。
 * Conversation Knowledge Graph Memory: 使用基于内存的知识库的形式，从用户输入中提取三元组，并保存，用于后续的问答。
 * ConversationSummaryMemory: 如果不指定对话轮数，而一味的将对话内容保存在memory中的话，很快就会因为超出token限制而出错。如果使用了window memory，则会丢弃老的对话记录。而使用这个ConversationSummaryMemory，他就会根据聊天历史生成summery，再将这个summery信息放到对话上下文中，用于后续对话。这样即使是很多轮对话，也不会超出token显示或丢失重要信息。
 * VectorStoreRetrieverMemory: 使用这个memory，他会将聊天历史保存在向量数据库中，然后每次对话的时候，根据用户输入查询最相近的几条记录，并作为上下文，回答用户问题。跟这个类似，也有很多将记录保存到各种数据库的memory。